In [ ]:
from kingdon import Algebra
import numpy as np
# from functools import wraps

# def tonp(func):
#     @wraps
#     def wrapped_func(*args, **kwargs):
#         return np.array(func(*args, **kwargs))
    # return func

# , wrapper=tonp
alg = Algebra(3, 0, 1)

Create a set of points to implement dynamics on.
- Do these need to have mass associated now?

In [ ]:
n = 10                             # Number of points
s = np.diag([0.1, 0.1, 0.1])       # Variance

points = np.random.multivariate_normal([0,0,0],s,n)
points = [alg.vector(e0=1, e1=points[i,0], e2=points[i,1], e3=points[i,2]).dual() for i in range(n)]


The physicis.

In [ ]:
# Gravitational force and spring constant
fg = alg.multivector(e02=-9.81)
spring_constant = 6
attach = alg.vector(e0=1, e1=0, e2=2, e3=0).dual()

def force(M, B, point):
    gravitational = (M.reverse() >> fg).dual()
    hooke = -spring_constant*((M.reverse() >> attach) & point)
    damping = (-0.75 * B).dual()
    return hooke + gravitational + damping



state = [
    [alg.evenmv(e=1, e01=0.0, e02=0.0, e12=0.0),           # The starting position and orientation in space. (identity) 
    alg.multivector(e01=0.0, e02=0.0, e12=-0.2)]           # The starting linear and rotational velocity in body.
    for i in range(n)
]

dState = lambda M, B, point: [                                   # Change in M and B
    -0.5 * M * B,
    (force(M, B, point) +  B.cp(B.dual())).undual()
]

def euler(state,dState,dt,i):
    return [state[i][0] + dState(*state[i], points[i])[0] * dt, state[i][1] + dState(*state[i], points[i])[1] * dt]

In [ ]:
def graph_func():
    global state

    for _ in range(10):
        state = [euler(state, dState, 1/600, i) for i in range(n)]

    return [
        0x007799,
        attach, "Fixed", 
        # 0x00DD88, *points,
        *[[attach, state[i][0] >> points[i]] for i in range(n)],
        *[state[i][0] >> points[i] for i in range(n)],
    ]

alg.graph(
    graph_func,
    animate=True,
    labels=True,
    grid=True,
    scale=0.5,
    lineWidth=6,
)

In [ ]:
# def graph_func():
#     return [
#         0x224488, f"Dynamic Points",
#         0x00DD88, *points,
#         # 0x00DD88, *[points[i] & points[i+1] for i in range(n-1)],
#     ]

# g = alg.graph(
#     graph_func, grid=True, lineWidth=6, labels=True,
# )
# g